# SKT-koBERT

https://github.com/SKTBrain/KoBERT

## Install & Library

In [1]:
# !nvidia-smi

In [2]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import pandas as pd
import numpy as np
from tqdm import tqdm, tqdm_notebook
import os

from KoBERT.kobert.utils import get_tokenizer
from KoBERT.kobert.pytorch_kobert import get_pytorch_kobert_model

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

import pickle

np.set_printoptions(formatter={'float_kind': lambda x: "{0:0.5f}".format(x)})

# CPU or GPU
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
device

'cuda:0'

## Path

In [3]:
model_path = './Model/'
base_path = '../Data/QnA/'

## Model & Tokenizer

In [4]:
bertmodel, vocab = get_pytorch_kobert_model()

using cached model
using cached model


In [5]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model


In [6]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i]) for i in dataset] # sent_idx를 제거

    def __getitem__(self, i):
        return (self.sentences[i])

    def __len__(self):
        return (len(self.sentences))

In [7]:
## Setting parameters
max_len = 210 #210 #수정
batch_size = 16 # 64 #수정
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [8]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=5, #2 #수정
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        
        classifier = self.classifier(pooler)
        
        if self.dr_rate:
            out = self.dropout(classifier)
  
        return pooler, out

In [9]:
# 모델 설정
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)

In [10]:
def question_BERTClassifier(_dataloader):  
    # 기존 코드가 실행이 너무 오래 걸려서 최대한 빠르게 돌아가도록 수정해보기

    # 저장할 빈 리스트 생성  
    question_embedding = []
    question_probability = []

    for batch_id, (token_ids, valid_length, segment_ids) in enumerate(tqdm_notebook(_dataloader)):

        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length  

        embedding, probability = model.forward(token_ids, valid_length, segment_ids)

        embedding = embedding.cpu().detach().numpy()
        probability = probability.cpu().detach().numpy()

        # list에 append
        question_embedding.append(embedding)
        question_probability.append(probability)

        del embedding, probability
        torch.cuda.empty_cache() # GPU 캐시 삭제
    
    # concat
    question_embedding_ = np.concatenate([que for que in question_embedding], axis=0)
    question_probability_ = np.concatenate([que for que in question_probability], axis=0)
    
    # max index 추출
    question_class = np.argmax(np.array(question_probability_), axis=1)
    
    return question_embedding_, question_class

## Question Embedding & Classification

In [11]:
# 모델 불러오기
model.load_state_dict(torch.load(model_path+'type1_KoBERT_new.pt'))
# 모델 evaluation으로
model.eval()

BERTClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True

In [12]:
# load data
df = pd.read_csv(base_path + 'qna_table_mecab_SN.csv')

# column drop
df = df.drop(columns=['question_mecab', 'answer_mecab', 'qna_hanspell_sep', 'qna_mecab_sep', 'question_embedding', 'question_class'])

# input list 생성
inputs = df['question_mecab_SN'].astype(str).tolist()

# make dataset
data = BERTDataset(inputs, 0, tok, max_len, True, False)
_dataloader = torch.utils.data.DataLoader(data, batch_size=batch_size, num_workers=5)

In [13]:
# 제대로 들어감 확인
data[2]

(array([   2, 4162, 6137, 3597, 5336, 1370, 6493,  517, 6056, 7483, 1732,
        6122, 2437, 1562, 4162, 6137, 2086, 3597, 5336, 1370, 6493,  517,
        6056, 7483, 4213, 2872, 3854,    3,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1, 

In [16]:
# embedding & classifier
question_embedding, question_class = question_BERTClassifier(_dataloader)

/compuworks/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  del sys.path[0]


  0%|          | 0/5730 [00:00<?, ?it/s]

In [21]:
question_embedding.shape, question_class.shape

((91672, 768), (91672,))

In [18]:
# dataframe 파일 생성
df_out = pd.DataFrame()
df_out['question_embedding'] = question_embedding.tolist()
df_out['question_class'] = question_class.tolist()

In [23]:
df_out[:1]

,question_embedding,question_class
0,"[-0.4953547418117523, -0.5073815584182739, 0.5...",1


In [24]:
# 분류 결과
df_out['question_class'].value_counts()

0    45075
1    30463
2     7744
3     6140
4     2250
Name: question_class, dtype: int64

In [26]:
# 기존 파일과 결합
df_=pd.merge(df,df_out,left_index=True,right_index=True)

In [28]:
print(df_.shape)

df_[:1]

(91672, 20)

## Save Data

In [32]:
# 용량을 줄이기 위해 pickle로 저장
with open(base_path+"question_embedding_SN_new.pkl", "wb") as file:
    pickle.dump(df_, file)